# VacationPy
----
#### Author: Amir Vazquez
Date: 12/31/2020
#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
try:
    import matplotlib.pyplot as plt
    from scipy.stats import linregress
    import pandas as pd
    import numpy as np
    import requests
    import gmaps
    import os, sys

    # Import API key
    sys.path.append('..\\apikeys')
    from api_keys import g_key
    
    #from my_func import lregression
    
    # Configure gmaps
    gmaps.configure(api_key=g_key)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
except Exception as e:
    print(f"a module(s) have not been imported {e}" )

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../starter_code/output_data/cities.csv")
city_weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa maria,-29.68,-53.81,18.60,82,72,2.73,BR,1609746832
1,1,dikson,73.51,80.55,-20.28,86,80,7.79,RU,1609746832
2,2,neepawa,50.23,-99.47,1.00,86,90,10.80,CA,1609746832
3,3,san quintin,30.48,-115.95,13.07,88,0,2.10,MX,1609746833
4,4,cherskiy,68.75,161.30,-33.24,85,99,2.17,RU,1609746833


In [3]:
#Drop column Unnamed:0
city_weather_df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
city_weather_df.dtypes 

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [5]:
city_weather_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,564.000000,564.000000,564.00000,564.000000,564.000000,564.000000,5.640000e+02
mean,21.048954,18.682571,9.78117,74.381206,59.051418,3.514255,1.609747e+09
std,33.362696,91.386952,17.73852,19.864507,39.380990,2.593409,7.889311e+01
min,-54.800000,-179.170000,-45.97000,1.000000,0.000000,0.050000,1.609746e+09
25%,-5.335000,-64.875000,0.42000,67.000000,20.000000,1.515000,1.609747e+09
50%,28.020000,23.140000,13.95500,80.000000,75.000000,2.740000,1.609747e+09
75%,48.162500,96.860000,25.00000,88.000000,95.000000,4.600000,1.609747e+09
max,78.220000,179.320000,38.68000,100.000000,100.000000,13.400000,1.609747e+09


### Humidity Heatmap
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# store values of location (latitude and longitude) into loc[] for use
loc = city_weather_df[["Lat","Lng"]]
# store values of humidity into hum[] for use
hum = city_weather_df["Humidity"].astype(float)


In [7]:
#Heatmap plot 
fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(loc, 
                                 weights = hum, 
                                 dissipating = False, 
                                 max_intensity = 200,
                                 point_radius = 4)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [8]:
# Remove rows missing data
city_df = city_weather_df

maxtempf = city_df['Max Temp']
windspeedmph = city_df['Wind Speed']

# Convert temperature from C to F and add a new column
city_df['Max Temp F'] = maxtempf*(9/5)+32
# Convert Wind Speed from Km to MPH and add a new column
city_df['Wind Speed (mph)'] = windspeedmph/1.609
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph)
0,santa maria,-29.68,-53.81,18.60,82,72,2.73,BR,1609746832,65.480,1.696706
1,dikson,73.51,80.55,-20.28,86,80,7.79,RU,1609746832,-4.504,4.841516
2,neepawa,50.23,-99.47,1.00,86,90,10.80,CA,1609746832,33.800,6.712244
3,san quintin,30.48,-115.95,13.07,88,0,2.10,MX,1609746833,55.526,1.305158
4,cherskiy,68.75,161.30,-33.24,85,99,2.17,RU,1609746833,-27.832,1.348664


In [9]:
# this combination will only pull 9 records
# city_df_upd = city_df.loc[(city_df['Max Temp F'] > 70) & (city_df['Max Temp F'] < 80)]
# city_df_upd = city_df_upd.loc[city_df_upd['Wind Speed (mph)']< 10 ]
# city_df_upd = city_df_upd.loc[city_df_upd['Cloudiness'] == 0]

# this combination will bring 35 potential vacation spots its ok if its a little hotter
city_df_upd = city_df.loc[(city_df['Max Temp F']>70) & (city_df['Max Temp F']< 90)]
city_df_upd = city_df_upd.loc[city_df_upd['Wind Speed (mph)']< 10 ]
city_df_upd = city_df_upd.loc[city_df_upd['Cloudiness']<2]
                          
city_df_upd.dropna()
city_df_upd.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Max Temp F,Wind Speed (mph)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,3.500000e+01,35.000000,35.000000
mean,0.903714,20.835143,26.131714,49.457143,0.114286,4.391143,1.609747e+09,79.037086,2.729113
std,21.101051,72.847812,2.696878,28.111483,0.322803,2.531320,8.917648e+01,4.854380,1.573226
min,-33.930000,-156.470000,22.000000,15.000000,0.000000,0.890000,1.609747e+09,71.600000,0.553139
25%,-20.800000,-5.155000,24.260000,19.000000,0.000000,2.555000,1.609747e+09,75.668000,1.587943
50%,10.370000,12.920000,26.210000,57.000000,0.000000,3.720000,1.609747e+09,79.178000,2.311995
75%,17.325000,70.490000,27.875000,77.000000,0.000000,6.040000,1.609747e+09,82.175000,3.753884
max,27.980000,167.480000,31.900000,86.000000,1.000000,12.300000,1.609747e+09,89.420000,7.644500


In [10]:
city_df_upd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph)
6,busselton,-33.65,115.33,28.89,29,0,0.89,AU,1609746712,84.002,0.553139
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,1609746835,79.232,1.162213
19,cidreira,-30.18,-50.21,22.05,78,0,7.31,BR,1609746836,71.690,4.543195
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,1609746839,77.666,3.971411
39,adre,13.47,22.20,26.21,17,0,6.77,TD,1609746841,79.178,4.207582


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Hotel variables
hotel_df = city_df_upd.copy()
hotel_df['Hotel Name']=''
hotel_df['Vicinity']=''
hotel_df['Rating']=''
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph),Hotel Name,Vicinity,Rating
6,busselton,-33.65,115.33,28.89,29,0,0.89,AU,1609746712,84.002,0.553139,,,
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,1609746835,79.232,1.162213,,,
19,cidreira,-30.18,-50.21,22.05,78,0,7.31,BR,1609746836,71.690,4.543195,,,
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,1609746839,77.666,3.971411,,,
39,adre,13.47,22.20,26.21,17,0,6.77,TD,1609746841,79.178,4.207582,,,


In [13]:
# geocoordinates
target_coordinates = "0, 0"
target_search = "hotel"
target_radius = 10000
#target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
 #   "type": target_type,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    #restr_type = row['ethnicity']
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    cityname = row["City"]
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {target_search} in {cityname} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Vicinity']   = results[0]['vicinity']
        #Add Hotel Rating to the df
        hotel_df.loc[index, 'Rating'] = results[0]['rating']
# for testing        
#        print(response["results"][0]["name"])
#        print(response["results"][0]["vicinity"])
#        print(response["results"][0]["rating"])
        
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping.")
        
    print("------------")
print("Search End ----------------------")    

Closest hotel in busselton is Esplanade Hotel.
------------
Closest hotel in san-pedro is ENOTEL BEACH.
------------
Closest hotel in cidreira is Hotel Navegantes.
------------
Closest hotel in rikitea is Pension Maro'i.
------------
Missing field/result... skipping.
------------
Closest hotel in kahului is Maui Beach Hotel.
------------
Closest hotel in port augusta is Sip'n Save - Pastoral Hotel.
------------
Closest hotel in cape town is Atlanticview Cape Town Boutique Hotel.
------------
Closest hotel in kulhudhuffushi is Nolhi Escape.
------------
Closest hotel in kontagora is City Gate Inn.
------------
Missing field/result... skipping.
------------
Closest hotel in axim is Ankobra Beach Resort.
------------
Missing field/result... skipping.
------------
Closest hotel in tamandare is Ancoradouro Hotel Tamandaré - PE.
------------
Closest hotel in saldanha is Club Mykonos Resort.
------------
Closest hotel in vao is Le Méridien Ile des Pins.
------------
Closest hotel in cockburn 

In [14]:
hotel_df.head(30)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed (mph),Hotel Name,Vicinity,Rating
6,busselton,-33.65,115.33,28.89,29,0,0.89,AU,1609746712,84.002,0.553139,Esplanade Hotel,"30-38 Marine Terrace, Busselton",4.1
11,san-pedro,4.75,-6.64,26.24,85,0,1.87,CI,1609746835,79.232,1.162213,ENOTEL BEACH,"Boulevard Houphouët-Boigny, San-Pédro",3.9
19,cidreira,-30.18,-50.21,22.05,78,0,7.31,BR,1609746836,71.690,4.543195,Hotel Navegantes,"R. Navegantes, 728 - Balneário Pinhal",4.2
31,rikitea,-23.12,-134.97,25.37,80,0,6.39,PF,1609746839,77.666,3.971411,Pension Maro'i,Gambier,4
39,adre,13.47,22.20,26.21,17,0,6.77,TD,1609746841,79.178,4.207582,,,
54,kahului,20.89,-156.47,23.00,78,1,6.20,US,1609746674,73.400,3.853325,Maui Beach Hotel,"170 W Kaahumanu Ave, Kahului",3.8
62,port augusta,-32.50,137.77,28.33,53,0,12.30,AU,1609746783,82.994,7.644500,Sip'n Save - Pastoral Hotel,"17 Stirling Rd, Port Augusta",4.3
87,cape town,-33.93,18.42,25.00,61,0,1.50,ZA,1609746619,77.000,0.932256,Atlanticview Cape Town Boutique Hotel,"31 Francolin Rd, Camps Bay, Cape Town",4.9
135,kulhudhuffushi,6.62,73.07,27.65,72,0,2.73,MV,1609746863,81.770,1.696706,Nolhi Escape,Nolhivaranfaru,3.5
137,kontagora,10.40,5.47,28.78,18,0,2.28,NG,1609746863,83.804,1.417029,City Gate Inn,"1 Federal Lowcost Housing Estate, Kontagora",3.7


In [15]:
# 3 records with NAN will be dropping these values
hotel_df= hotel_df.dropna()
#Some records do not come back with NaN well search for blank
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] !=''] 
hotel_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Max Temp F,Wind Speed (mph)
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,2.600000e+01,26.000000,26.000000
mean,-4.818462,18.724231,26.446154,58.961538,0.153846,4.628846,1.609747e+09,79.603077,2.876847
std,21.516687,82.814023,2.789817,25.558530,0.367946,2.801450,9.503904e+01,5.021671,1.741113
min,-33.930000,-156.470000,22.000000,18.000000,0.000000,0.890000,1.609747e+09,71.600000,0.553139
25%,-28.415000,-30.282500,24.477500,34.750000,0.000000,2.350000,1.609747e+09,76.059500,1.460534
50%,4.810000,11.705000,26.435000,69.500000,0.000000,4.040000,1.609747e+09,79.583000,2.510876
75%,11.572500,81.102500,28.257500,79.500000,0.000000,6.342500,1.609747e+09,82.863500,3.941889
max,27.850000,167.480000,31.900000,86.000000,1.000000,12.300000,1.609747e+09,89.420000,7.644500


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#print(hotel_info)

In [17]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, 
                                 info_box_content=hotel_info)
# Display figure
#Heatmap plot 
fig1.add_layer(hotel_layer)

location_marker = gmaps.marker_layer(locations)
#fig1.add_layer(heat_layer)
fig1.add_layer(location_marker)

fig1

Figure(layout=FigureLayout(height='420px'))